In [ ]:
!pip install --upgrade pip
!pip install transformers accelerate datasets pillow peft bitsandbytes torchvision torchaudio gradio

In [7]:
!pip install kagglehub kaggle

In [8]:
import kaggle

# Download dataset from Kaggle

path = kaggle.api.dataset_download_files("adityajn105/flickr8k", unzip=True, path="./data")

print(f"Dataset downloaded and extracted to: {path}")

Dataset URL: https://www.kaggle.com/datasets/adityajn105/flickr8k
Dataset downloaded and extracted to: None


Below is just imports and setting threads

In [ ]:
import os, json, random, time, math
from pathlib import Path

import torch

intensity = 5  # Scale of 1-10 for CPU usage
torch.set_num_threads(max(1, int(os.cpu_count() // (1 / (intensity / 10))))) # Sets the number of threads to the cpu_count divided by the reciprocal of the intensity(scaled 1 - 10) divided by 10

from PIL import Image
from datasets import load_dataset, Dataset

from transformers import (

    AutoTokenizer,
    AutoProcessor,
    BlipForConditionalGeneration,
    TrainingArguments,
    Trainer,
    default_data_collator,

)


Ok, so basically I am on CPU, so I'll put a snippet of code below that reduces the size of the dataset and processes it.

In [ ]:

# Paths -> Data, Images, Captions.txt; Replace your paths here

DATA_DIR = Path("data")
IMAGES_DIR = DATA_DIR / "Images"
CAPTIONS_FILE = DATA_DIR / "captions.txt"

# Make sure nothing is missing, everything exists, etc.

assert IMAGES_DIR.exists(), f"Images directory {IMAGES_DIR} does not exist. Run the cells above."
assert CAPTIONS_FILE.exists(), f"Captions file {CAPTIONS_FILE} does not exist. Run the cells above."

# Function: 

def parse_captions(path: Path):

    """
    
    put ts explanation here
    
    """

